# Simple Chat Bot

* En este proyecto personal vamos a programar un chat bot, que va a contestar algunas preguntas que se hacen con frecuencia, con un enfoque para un negocio. 
* Como va a ser un chat bot simple no vamos a descargar ningun dataset, sino que vamos a escribir los datos de entrenamiento del modelo nosotros mismos
* La idea es hacer un chat bot de la siguiente manera: 1) El usuario ingrese una pregunta, 2) Hacer una prediccion con el modelo para ver a cual clase de pregunta hace referencia 3) Elegir alguna respuesta de esa misma clase al azar
* Explicamos sin entrar en mucho detalle cada parte del codigo

### Los datos de entramiento son los siguientes: 

# Empezamos con el proyecto

Importamos las librerias necesarias e importamos nuestro json con los datos de entrenamiento

In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
from nltk import SnowballStemmer
nltk.download('punkt')
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import keras
import pickle


import json
with open('intents_es.json') as file:
    data = json.load(file)

[nltk_data] Downloading package punkt to /Users/ramiro/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


### Extraemos la data necesaria y la vamos ordenando para su uso

In [2]:
words = []
labels = []
docs_x = []
docs_y = []

In [3]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

### Stemming

Usamos Stemming para llevar las palabras a su raiz, esto ayuda al entrenamiento del modelo.

In [4]:
spanishstemmer=SnowballStemmer('spanish')
words = [spanishstemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))
labels = sorted(labels)

### Modelo bolsa de palabras

Sabemos que las redes neuronales necesitan numeros para su entramiento para eso usamos bag of words o bolsa de palabras.

In [5]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [6]:
training = numpy.array(training)
output = numpy.array(output)

### Creamos nuestro modelo con tflearn

Vamos a usar como modelo una red neuronal con dos capas ocultas full connected, la idea es que lea la bolsa de palabras (input) y nos diga a que clase corresponde cada pregunta (output)

In [7]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)

Training Step: 3999  | total loss: 0.28998 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.28998 - acc: 0.8980 -- iter: 24/32
Training Step: 4000  | total loss: 0.31998 | time: 0.012s
| Adam | epoch: 1000 | loss: 0.31998 - acc: 0.8832 -- iter: 32/32
--


### Usamos el bot

* Queremos que gerene una respuesta a la pregunta ingresada por el usuario, nuestro modelo nos va a dar una lista de probabilidades para todas nuestras clases, de la clase con la probabilidad mas alta elegimos una respuestesta al azar.
* Si la probabilidad mas alta no supera el 70% vamos a decirle al usuario que intente de nuevo, ya que no tenemos una respuesta contundente a esa pregunta.

In [8]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


No_entiendo = ["No entiendo, intenta de nuevo", "No estoy preparado para responder eso :(", "No lo entiendo"]

def chat():
    print("Empezá a chatear con el bot (escriba salir para terminar)")
    while True:
        inp = input("Vos: ")
        if inp.lower() == "salir":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        if results[results_index] > 0.70:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print(random.choice(responses))
            
        else:
            print (random.choice(No_entiendo))

chat()

Empezá a chatear con el bot (escriba salir para terminar)
Vos: Hola
¡Qué bueno verte de nuevo!
Vos: Como te llamas?
Puedes llamarme Ramiro
Vos: que edad tenes?
22 años gracias por preguntar!
Vos: Me gustaría comprar
¡Las galletas están en el menú!
Vos: cual es su horario
¡Abrimos de 7 am a 4 pm de lunes a viernes!
Vos: hasta luego
¡Adiós!
Vos: salir
